In [272]:
import pandas as pd
import plotly as pl

from plotly import express as px

df = pd.read_csv('resources/all_stocks_5yr.csv', parse_dates=[0])
df['Weekday'] = df['date'].apply(lambda x: x.isoweekday())
df['Weeknum'] = df['date'].dt.isocalendar().week
df['Year'] = df['date'].dt.isocalendar().year

In [273]:
#- Could you show us how the median `close` stock price changed year over year? Use whichever visualization you would fine suitable.

df1 = df.groupby('date')[["close"]].median().reset_index()

fig = px.line(df1, x='date', y='close', labels={"close":"Daily median close price"}, title="Median close price S&P (YoY)")
fig.update_layout(xaxis=dict(tick0='2013-02-08', dtick=(86400000.0 * 365),
tickmode='linear'))
fig.show()

In [274]:
#- How did the volatility of this stock index changed year over year? For the sake of exercise, 
# don't bother with VIX and similar proper volatility indicators - a standard deviation of `close` price will do. What do you see?

df1 = df1.groupby(df1['date'].dt.year)[["close"]].std().reset_index()
df1

#I have calculated Volatility applying std to the yearly grouped median daily prices calculated in df above. 
# From the output we can conclude that the S&P500 index price is quite stable and got even better in 2017-2018.
# We can observe high volatility during financial crises years (like 2008) and lots of outliers come up.

,date,close
0,2013,2.609844
1,2014,2.608262
2,2015,2.280792
3,2016,3.382926
4,2017,1.934025
5,2018,1.596427


In [275]:
#- Which are the top 5 most volatile stocks in a year in that dataset? Group by `date`'s year and `name`, 
# use standard deviation to approximate volatility.

df3 = df.groupby([df['Year'],df['Name']])[["close"]].std().reset_index()
df3 = df3.sort_values(by='close',ascending=False)
df3 = df3.groupby('Year').head(5)
df3 = df3.sort_values(by='Year',ascending=True)

fig = px.histogram(df3, x='Year', y='close', color="Name", barmode='group', barnorm=None, title="Top 5 most volatile stocks S&P (YoY)", labels={"Name":"Stock","close":"Daily close price Std"})
fig.update_layout(bargap=0)
fig.show()

In [276]:
#- Let's zoom in and look at the Google stock in 2015 (`name` is `GOOGL`), again looking at the closing price.
#Could you show daily, weekly, and monthly dynamics? Weeks should start on Mondays and be labelled by Monday's date.

df4 = df.loc[(df['Name'] == 'GOOGL') & (df['date'].dt.year == 2015)]

#daily
fig = px.line(df4, x='date', y='close', title="GOOGL 'close' price 2015 S&P (DoD)", labels={"close":"Daily close price"})
fig.show()

#weekly
df5 = df4.loc[(df['Weekday'] == 1)]
df6 = df5.groupby(['Weeknum','date'])[["close"]].median().reset_index()
fig = px.line(df6, x='date', y='close', title="GOOGL 'close' price 2015 S&P (WoW)", labels={"close":"Weekly median close price",'date':'Week'})
fig.update_layout(xaxis=dict(tick0='2015-01-05', dtick=604800000.0,
tickmode='linear'))
fig.show()

#monthly
df7 = df4.groupby(df4['date'].dt.month)[["close"]].median().reset_index()
mt_mapping={
    1: 'Jan', 
    2: 'Feb', 
    3: 'Mar', 
    4: 'Apr', 
    5: 'May',
    6: 'Jun', 
    7: 'Jul',
    8: 'Aug',
    9: 'Sep',
    10: 'Oct',
    11: 'Nov',
    12: 'Dec'
} 
df7['Month'] = df7['date'].map(mt_mapping)
fig = px.line(df7, x='Month', y='close', title="GOOGL 'close' price 2015 S&P (MoM)", labels={"close":"Monthly median close price"})
fig.show()

#there might be a schrodinger's cat in a sealed weekly graph, lol

In [277]:
#- Finally, please, show the closing prices of Google, Amazon (`AMZN`), and Microsoft (`MSFT`) on the same chart,
#for the whole duration of data. What do you see?

df8 = df.loc[(df['Name'] == 'GOOGL') | (df['Name'] == 'AMZN') | (df['Name'] == 'MSFT')]

fig = px.line(df8, x='date', y='close', color="Name", title="GOOGL, AMZN & MSFT 'close' price S&P", labels={"close":"Daily close price", "Name":"Stock"})
fig.show()

#From the graph we can see that MSFT is quite cheap and stable comparing to GOOGL and AMZN.